### zad4

In [109]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from typing import List, Dict, Tuple
import torch
import numpy as np
from torch.nn import functional as F
from typing import List
import random
from collections import defaultdict

PAPUGA = 'flax-community/papuGaPT2'
DEVICE = "cuda"

tokenizer = AutoTokenizer.from_pretrained(PAPUGA)
model = AutoModelForCausalLM.from_pretrained(PAPUGA).to(DEVICE)
model.device

device(type='cuda', index=0)

In [110]:
TOKENS_LEN = 50257

all_tokens = [tokenizer.decode([idx]) for idx in range(TOKENS_LEN)]
all_tokens

['<s>',
 '<pad>',
 '</s>',
 '<unk>',
 '<mask>',
 '!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '{',
 '|',
 '}',
 '~',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�

In [111]:
prefixes: List[str] = []
with open("prefiksy.txt") as file:
    for line in file:
        prefixes.append(line.strip())

prefixes.sort() 
prefixes

['A A A.',
 'A Adria a',
 'A Antoni Antczak',
 'A Aslan A.',
 'A autorem artykułu,',
 'A.A. Ananin, A.I.',
 'A.A. Ananin, A.I.',
 'A.A. Ananin, A.I.',
 'AAF: Asociación Argentina',
 'AAm: Asociación Amateurs',
 'ABS, alufelgi, autoalarm,',
 'ABS, alufelgi, autoalarm,',
 'ADLER AFI ALBA',
 'AF: Ansel Adams',
 'AKCYZA AKCYZA AKCYZA',
 'AKL AGN AGU',
 'ALRIGHT, Abandon a',
 'Aa ab ac',
 'Aac, aif, aifc,',
 'Aaksens, Abbegaasterketting, Anneburen,',
 'Aaltonen, Adidas, Alpina,',
 'Aaltonen, Adidas, Alpina,',
 'Aarne Armas Arvonen',
 'Abada aldeia axe',
 'Abakanowicz abazur abc',
 'Abakanowicz abazur abc',
 'Abakanowicz abazur abc',
 'Abakanowicz abazur abc',
 'Abakanowicz abazur abc',
 'Abakanowicz abazur abc',
 'Abakanowicz abazur abc',
 'Abbotsford, Albourne, Ansty,',
 'Abbotskerswell, Ashcombe, Ashton,',
 'Abbotswood, Albury, Artington,',
 'Abchaski, afaryjski, afrikaan,',
 'Aber auch auf',
 'Aberford, Adwalton, Allerton',
 'Abrahamowicz, Abramowicz, Abrahimowicz,',
 'Abrahamowicz, Abra

In [112]:
class Generator: 
    def __init__(self, letter = 'p') -> None:
        self.valid_prefixes: List[str] = [prefix for prefix in prefixes if prefix.strip().lower().startswith(letter)]
        self.invalid_token_ids: List[int] = []
        
        for i in range(TOKENS_LEN):
            token: str = tokenizer.decode([i])
            if (token.startswith(" ") or token.startswith("\n") or token.startswith(",")) and not token.strip().lower().startswith(letter):
                self.invalid_token_ids.append(i)
                
        self.invalid_token_ids_tensor = torch.tensor(self.invalid_token_ids, device=DEVICE)
        
    def _generate(self, prompt: str, k: int, p: float, max_new_tokens: int = 25) -> str:
        input_ids = tokenizer(prompt, return_tensors="pt").to(DEVICE)["input_ids"]
        
        for _ in range(max_new_tokens):
            with torch.no_grad():
                output = model(input_ids=input_ids)
                
                logits = output.logits[:, -1, :]
                logits[:, self.invalid_token_ids_tensor] = -float('inf')
                probs = F.softmax(logits, dim=-1)[0]
                
                ok_tokens_ids, ok_tokens_probs = Generator._apply_top_pk(probs, k, p)
                next_token_id = np.random.choice(ok_tokens_ids, p=ok_tokens_probs)
                next_token_tensor = torch.tensor([[next_token_id]], device=DEVICE)
                
                input_ids = torch.cat([input_ids, next_token_tensor], dim=-1)
                
                if next_token_id == tokenizer.eos_token_id:
                    break
        
        return tokenizer.decode(input_ids[0], skip_special_tokens=True)[len(prompt) + 1:].capitalize()
    
    def generate(self, k: int = 3, p: float = 0.9, no_generations: int = 5) -> str:
        prompt = self._create_prompt()
        outputs = [self._generate(prompt, k, p) for _ in range(no_generations)] 
        outputs = [(Generator._eval(output), output) for output in outputs]
        return max(outputs)[1]
        
    @staticmethod
    def _eval(output: str) -> Tuple[float, str]:
        counter = defaultdict(int)
        for s in output.replace(".", " ").replace(",", " ").lower().split(" "):
            counter[s] += 1
            
        ans = 0
        for key, val in counter.items():
            ans -= val * val * val
            
        ans -= 20 * output.count(":")
        ans -= 10 * output.count("-")
        
        return (ans, output)
    
    @staticmethod
    def _apply_top_pk(probs: torch.Tensor, k: int, p: float) -> Tuple[List[int], List[float]]:
        sorted_probs, sorted_indicies = torch.sort(probs, descending=True)
        
        # top k
        sorted_probs, sorted_indicies = sorted_probs[:k], sorted_indicies[:k]
        
        # top p
        probs_sum = sorted_probs[0].item()
        end_idx = 0
        
        for i in range(1, k):
            if probs_sum >= p:
                break
            end_idx = i 
            probs_sum += sorted_probs[i].item()
            
        sorted_probs, sorted_indicies = sorted_probs[:end_idx + 1], sorted_indicies[:end_idx + 1]
        
        # renormalisation
        probs_np = np.array(sorted_probs.cpu().tolist(), dtype=np.float64)
        probs_np = probs_np / np.sum(probs_np)
        return sorted_indicies.tolist(), probs_np.tolist()
        
    def _create_prompt(self) -> str:
        few_shots = random.choices(self.valid_prefixes, k = 10)
        return " \n ".join([shot for shot in few_shots])

In [113]:
generator = Generator()
print(generator.generate(no_generations=45))

Pan poseł poseł pogoda powiedział: posłowie powinni przyjąć poprawkę poselską. posłowie przyjmowali poprawki poselskie. popraw
